In [ ]:
!apt-get install -y libgmp-dev libmpfr-dev libmpc-dev
!pip install cysignals
!pip install fpylll

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgmp-dev is already the newest version (2:6.2.1+dfsg-3ubuntu1).
libmpc-dev is already the newest version (1.2.1-2build1).
libmpfr-dev is already the newest version (4.1.0-3build3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install pulp

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatusOptimal, value
from fpylll import IntegerMatrix, LLL

def lll_reduce_matrix(A):
    """
    Applies LLL reduction to matrix A using fpylll.
    A: list of lists of integers (matrix)
    Returns: LLL-reduced version of A
    """
    m, n = len(A), len(A[0])
    mat = IntegerMatrix(m, n)
    for i in range(m):
        for j in range(n):
            mat[i, j] = A[i][j]

    LLL.reduction(mat)
    A_reduced = [[mat[i, j] for j in range(n)] for i in range(m)]
    return A_reduced

def solve_lwe_lll_milp(A, b, q, x_bounds, error_bound, initial_x=None, use_lll=True):
    """
    Solves an LWE instance via MILP, with optional LLL reduction.
    """
    # Apply LLL reduction if requested
    if use_lll:
        A = lll_reduce_matrix(A)

    m = len(A)
    n = len(A[0])

    prob = LpProblem("LWE_MILP_LLL", LpMinimize)

    # Secret x_j variables
    x_min, x_max = x_bounds if isinstance(x_bounds, tuple) else (None, None)
    x = []
    for j in range(n):
        var = LpVariable(f"x_{j}", cat="Integer", lowBound=x_min, upBound=x_max)
        x.append(var)
        if initial_x is not None:
            var.setInitialValue(initial_x[j])

    # Integer slack for modulus
    k = [LpVariable(f"k_{i}", cat="Integer") for i in range(m)]

    # Error decomposition and absolute value terms
    e_plus  = [LpVariable(f"e_plus_{i}",  lowBound=0) for i in range(m)]
    e_minus = [LpVariable(f"e_minus_{i}", lowBound=0) for i in range(m)]
    t       = [LpVariable(f"t_{i}",       lowBound=0) for i in range(m)]

    # Objective
    prob += lpSum(t)

    # Constraints
    for i in range(m):
        # A_i x + q k_i - b_i = e⁺ - e⁻
        prob += (
            lpSum(A[i][j] * x[j] for j in range(n)) +
            q * k[i] - b[i] == e_plus[i] - e_minus[i]
        )
        prob += t[i] >= e_plus[i]
        prob += t[i] >= e_minus[i]
        prob += e_plus[i] <= error_bound
        prob += e_minus[i] <= error_bound

    # Solve the MILP
    prob.solve()

    if prob.status == LpStatusOptimal:
        x_sol = [value(var) for var in x]
        errors = [value(e_plus[i]) - value(e_minus[i]) for i in range(m)]
        total_error = sum(value(t[i]) for i in range(m))
        return x_sol, errors, total_error
    else:
        return None, None, None


In [ ]:
# Example usage
if __name__ == "__main__":
    A = [[1, 2], [3, 4], [5, 6]]
    b = [3, 2, 1]
    q = 5
    x_bounds = (-1, 1)
    error_bound = 2

    x_sol, errors, total_error = solve_lwe_lll_milp(A, b, q, x_bounds, error_bound, use_lll=True)
    if x_sol is not None:
        print("Secret x:", x_sol)
        print("Per-coordinate errors:", errors)
        print("Total error:", total_error)
    else:
        print("No optimal solution found.")

Secret x: [1.0, 1.0]
Per-coordinate errors: [2.0, -1.0, 1.0]
Total error: 4.0


In [ ]:
import numpy as np

def modify_b_and_solve_small_steps(A, b, q, x_bounds, error_bound, step=0.01, range_val=2.0):
    b = np.array(b, dtype=float)
    perturb_range = np.arange(-range_val, range_val + step, step)

    for idx in range(len(b)):
        print(f"\n--- Perturbing b[{idx}] ---")
        for delta in perturb_range:
            b_modified = b.copy()
            b_modified[idx] += delta

            x_sol, errors, total_error = solve_lwe_lll_milp(A, b_modified.tolist(), q, x_bounds, error_bound, use_lll=True)

            if x_sol is not None:
                print(f"b = {b_modified}, x = {x_sol}")
            else:
                print(f"Delta = {delta:+.2f} -> No optimal solution")

# Example usage
if __name__ == "__main__":
    A = [[1, 2], [3, 4], [5, 6]]
    b = [3, 2, 1]
    q = 5
    x_bounds = (-1, 1)
    x_true = [1, 1]
    error_bound = 2

    modify_b_and_solve_small_steps(A, b, q, x_bounds, error_bound, step=0.05, range_val=2.0)



--- Perturbing b[0] ---
b = [1. 2. 1.], x = [1.0, 1.0]
b = [1.05 2.   1.  ], x = [1.0, 1.0]
b = [1.1 2.  1. ], x = [1.0, 1.0]
b = [1.15 2.   1.  ], x = [1.0, 1.0]
b = [1.2 2.  1. ], x = [1.0, 1.0]
b = [1.25 2.   1.  ], x = [1.0, 1.0]
b = [1.3 2.  1. ], x = [1.0, 1.0]
b = [1.35 2.   1.  ], x = [1.0, 1.0]
b = [1.4 2.  1. ], x = [1.0, 1.0]
b = [1.45 2.   1.  ], x = [1.0, 1.0]
b = [1.5 2.  1. ], x = [1.0, 1.0]
b = [1.55 2.   1.  ], x = [1.0, 1.0]
b = [1.6 2.  1. ], x = [1.0, 1.0]
b = [1.65 2.   1.  ], x = [1.0, 1.0]
b = [1.7 2.  1. ], x = [1.0, 1.0]
b = [1.75 2.   1.  ], x = [1.0, 1.0]
b = [1.8 2.  1. ], x = [1.0, 1.0]
b = [1.85 2.   1.  ], x = [1.0, 1.0]
b = [1.9 2.  1. ], x = [1.0, 1.0]
b = [1.95 2.   1.  ], x = [1.0, 1.0]
b = [2. 2. 1.], x = [1.0, 1.0]
Delta = -0.95 -> No optimal solution
Delta = -0.90 -> No optimal solution
Delta = -0.85 -> No optimal solution
Delta = -0.80 -> No optimal solution
Delta = -0.75 -> No optimal solution
Delta = -0.70 -> No optimal solution
Delta = -0.65 -

In [ ]:
# Example usage
if __name__ == "__main__":
    A = [[2, -1],       # 3x2 matrix
     [0, 3],
     [-2, 4]]
    b = [-1, 3, 4]
    q = 5
    # Suppose x_j in {-1,0,1}
    x_bounds = (-1, 1)
    error_bound = 2

    x_sol, errors, total_error = solve_lwe_lll_milp(A, b, q, x_bounds, error_bound)
    if x_sol is not None:
        print("Secret x:", x_sol)
        print("Per-coordinate errors:", errors)
        print("Total error:", total_error)
    else:
        print("No optimal solution found.")

Secret x: [-1.0, 0.0]
Per-coordinate errors: [1.0, 0.0, -1.0]
Total error: 2.0


In [ ]:
import numpy as np

def modify_b_and_solve_small_steps(A, b, q, x_bounds, error_bound, step=0.01, range_val=2.0):
    b = np.array(b, dtype=float)
    perturb_range = np.arange(-range_val, range_val + step, step)

    for idx in range(len(b)):
        print(f"\n--- Perturbing b[{idx}] ---")
        for delta in perturb_range:
            b_modified = b.copy()
            b_modified[idx] += delta

            x_sol, errors, total_error = solve_lwe_lll_milp(A, b_modified.tolist(), q, x_bounds, error_bound)

            if x_sol is not None:
                print(f"b = {b_modified}, x = {x_sol}")
            else:
                print(f"Delta = {delta:+.2f} -> No optimal solution")

if __name__ == "__main__":
    A = [[2, -1],       # 3x2 matrix
     [0, 3],
     [-2, 4]]
    b = [-1, 3, 4]
    q = 5
    # Suppose x_j in {-1,0,1}
    x_bounds = (-1, 1)
    error_bound = 2

    modify_b_and_solve_small_steps(A, b, q, x_bounds, error_bound, step=0.05, range_val=2.0)



--- Perturbing b[0] ---
b = [-3.  3.  4.], x = [-1.0, 0.0]
Delta = -1.95 -> No optimal solution
Delta = -1.90 -> No optimal solution
Delta = -1.85 -> No optimal solution
Delta = -1.80 -> No optimal solution
Delta = -1.75 -> No optimal solution
Delta = -1.70 -> No optimal solution
Delta = -1.65 -> No optimal solution
Delta = -1.60 -> No optimal solution
Delta = -1.55 -> No optimal solution
Delta = -1.50 -> No optimal solution
Delta = -1.45 -> No optimal solution
Delta = -1.40 -> No optimal solution
Delta = -1.35 -> No optimal solution
Delta = -1.30 -> No optimal solution
Delta = -1.25 -> No optimal solution
Delta = -1.20 -> No optimal solution
Delta = -1.15 -> No optimal solution
Delta = -1.10 -> No optimal solution
Delta = -1.05 -> No optimal solution
b = [-2.  3.  4.], x = [-1.0, 0.0]
b = [-1.95  3.    4.  ], x = [-1.0, 0.0]
b = [-1.9  3.   4. ], x = [-1.0, 0.0]
b = [-1.85  3.    4.  ], x = [-1.0, 0.0]
b = [-1.8  3.   4. ], x = [-1.0, 0.0]
b = [-1.75  3.    4.  ], x = [-1.0, 0.0]
b =